von Fabian Wolter und Selin Kabak
# Kalibrierung von Digitalkameras
### Versuch 2, Praktikum Technische Grundlagen der angewandten Informatik

TODO (1) Aufnahme und Analyse eines Grauwertkeiles
: 
- Speichern Sie das Bild für die Weiterverarbeitung und für das Notebook (Befehl: cv.imwrite()) im verlustfreien Format png ab.
- Schreiben Sie eine Routine, die das Bild einliest (Befehl: pyopencv.imread()) und die einzelnen Grauwertstufen als Unterbilder aus dem Originalbild ausliest.
- Bestimmen Sie für jede Stufe den Mittelwert und die Standardabweichung der Grauwerte und tragen Sie diese in eine Tabelle ein.

In [19]:
%pip install opencv-python
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(0)

# capture frames from the camera
# while True:
#     ret, frame = cap.read()

#     # Convert the frame to grayscale
#     grayscale = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#     cv.imshow('Grayscale', grayscale)

#     # Break the loop if 'q' is pressed
#     if cv.waitKey(1) & 0xFF == ord('q'):
#         # Save 10 images after exiting the loop
#         for i in range(10):
#             ret, frame = cap.read()  # Read another frame
#             grayscale = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#             cv.imwrite(f'weiss/weissbild{i}.png', grayscale)
#             cv.waitKey(1)  # Brief delay
#         break

# Release the camera and close any OpenCV windows
cap.release()
cv.destroyAllWindows()

# Number of grayscale levels
num_levels = 10
level_stats = []

# Read each saved image to calculate the mean and standard deviation for each sub-image
for i in range(10):
    image = cv.imread(f'Kamera2/weiss/weissbild{i}.png', cv.IMREAD_GRAYSCALE)
    # Split the image into num_levels sub-images based on height
    level_height = image.shape[0] // num_levels  # Calculate height for each level
    for j in range(num_levels):
        # Extract sub-image for each gray level
        sub_image = image[j * level_height: (j + 1) * level_height, :]

        # Calculate mean and standard deviation
        mean = np.mean(sub_image)
        stddev = np.std(sub_image)
        level_stats.append((mean, stddev))

for i, (mean, stddev) in enumerate(level_stats):
    image_num = i // num_levels + 1
    level_num = i % num_levels + 1
    print(f'Image {image_num}, Level {level_num}: Mean = {mean:.2f}, StdDev = {stddev:.2f}')

Note: you may need to restart the kernel to use updated packages.
Image 1, Level 1: Mean = 155.73, StdDev = 3.62
Image 1, Level 2: Mean = 156.68, StdDev = 3.99
Image 1, Level 3: Mean = 157.95, StdDev = 4.36
Image 1, Level 4: Mean = 159.20, StdDev = 4.74
Image 1, Level 5: Mean = 160.22, StdDev = 5.09
Image 1, Level 6: Mean = 161.66, StdDev = 5.54
Image 1, Level 7: Mean = 162.38, StdDev = 5.75
Image 1, Level 8: Mean = 162.85, StdDev = 6.12
Image 1, Level 9: Mean = 163.88, StdDev = 6.32
Image 1, Level 10: Mean = 163.92, StdDev = 6.57
Image 2, Level 1: Mean = 155.76, StdDev = 3.61
Image 2, Level 2: Mean = 156.66, StdDev = 3.97
Image 2, Level 3: Mean = 157.92, StdDev = 4.37
Image 2, Level 4: Mean = 159.23, StdDev = 4.72
Image 2, Level 5: Mean = 160.17, StdDev = 5.10
Image 2, Level 6: Mean = 161.63, StdDev = 5.51
Image 2, Level 7: Mean = 162.33, StdDev = 5.72
Image 2, Level 8: Mean = 162.77, StdDev = 6.12
Image 2, Level 9: Mean = 163.92, StdDev = 6.32
Image 2, Level 10: Mean = 163.96, StdDev

TODO (2) Aufnahme eines Dunkelbildes
: 
- Schreiben Sie eine Routine, die die 10 Bilder einliest, in double umwandelt, die Farbbilder in Grauwertbilder umrechnet und ihren pixelweisen Mittelwert berechnet. Stellen Sie das Dunkelbild kontrastmaximiert in Ihrem Notebook dar.
- Schreiben Sie eine Routine, die das Dunkelbild einliest, von einem Eingabebild subtrahiert und das korrigierte Bild abspeichert.

In [20]:
grayscale_images = []

for i in range(10):
    img = cv.imread(f'Kamera2/schwarz/schwarzbild{i}.png')
    grayscale_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY).astype(float)  # Convert to grayscale
    grayscale_images.append(grayscale_img)  # Store in the list

mean_image = np.mean(grayscale_images, axis=0)

# contrast maximized
normalized_mean_image = mean_image / np.max(mean_image)
cv.imshow('Mean Image (Normalized)', normalized_mean_image)
cv.waitKey(0)  # Wait for a key press to close the window
cv.destroyAllWindows()

cv.imwrite('dark_image.png', mean_image.astype(np.uint8))

# Subtract the Dark Image from an Input Image
def subtract_dark_image(input_image_path, dark_image_path, output_image_path):
    input_image = cv.imread(input_image_path).astype(float)
    dark_image = cv.imread(dark_image_path).astype(float)

    # Subtract the dark image from the input image
    corrected_image = cv.subtract(input_image, dark_image)

    # Convert corrected image to uint8
    corrected_image_uint8 = np.clip(corrected_image, 0, 255).astype(np.uint8)
    cv.imwrite(output_image_path, corrected_image_uint8)

subtract_dark_image('Kamera2/frame.png', 'dark_image.png', 'corrected_image.png')

TODO (3) Aufnahme eines Weissbildes
: 
- Schreiben Sie eine Routine, die die 10 Bilder einliest und ihren Mittelwert berechnet. Subtrahieren Sie von dem Mittelwertbild das Dunkelbild und speichern Sie das resultierende Weissbild ab. Stellen Sie das Weissbild kontrastmaximiert dar.
- Erweitern Sie die Routine zur Subtraktion des Dunkelbildes aus Aufgabe 2 so, dass das Weissbild eingelesen, in double umgewandelt wird und so normiert wird, dass sein Mittelwert 1 ist. Das durch Abzug des Dunkelbildes korrigierte Eingangsbild wird anschliessend durch das normierte Weissbild dividiert. 

In [21]:
def calculate_white_image(white_images_path, dark_image_path, output_path):
    imgarr = []
    for i in range(10):
        img = cv.imread(f"{white_images_path}/weissbild{i}.png", cv.IMREAD_GRAYSCALE).astype(np.double)
        imgarr.append(img)

    mean_image = np.mean(imgarr, axis=0)

    dark_image = cv.imread(dark_image_path, cv.IMREAD_GRAYSCALE).astype(np.double)

    # Calculate white image
    white_image = mean_image - dark_image
    cv.imwrite(f"{output_path}/white_image.png", white_image.astype(np.uint8))

    # contrast maximized
    normalized_white_image = white_image / np.max(white_image)
    cv.imshow('White Image', normalized_white_image)
    cv.waitKey(0)  # Keep the window open until a key is pressed
    cv.destroyAllWindows()
    return white_image

# correct an input image by subtracting the dark image and dividing by the white image
def correct_image(input_image_path, dark_image_path, white_image, output_path):
    input_image = cv.imread(input_image_path, cv.IMREAD_GRAYSCALE).astype(np.double)
    dark_image = cv.imread(dark_image_path, cv.IMREAD_GRAYSCALE).astype(np.double)

    corrected_image = (input_image - dark_image) / white_image

    # Clip to [0, 255] and convert to uint8
    corrected_image_uint8 = np.clip(corrected_image, 0, 255).astype(np.uint8)
    cv.imwrite(f"{output_path}/corrected_image.png", corrected_image_uint8)

# Paths
white_images_path = "Kamera2/weiss"
dark_image_path = "dark_image.png"
output_path = "Kamera2/changed"

# Calculate the white image
white_image = calculate_white_image(white_images_path, dark_image_path, output_path)

# example
correct_image("Kamera2/frame.png", dark_image_path, white_image, output_path)

TODO (4) Pixelfehler
: 
- Korrigieren Sie mithilfe Ihres Programms aus Aufgabe 3 das Bild des Grauwertkeils aus Aufgabe1.
- Werten Sie nun erneut das korrigierte Bild aus Aufgabe4 mit diesem Programm aus und erstellen Sie die zugehoerige Tabelle. Ergibt sich eine Verbesserung?

In [22]:
# mark hot pixels or dead pixels
def mark_pixels(image_path, threshold, color):
    image = cv.imread(image_path, cv.IMREAD_GRAYSCALE).astype(np.double)

    # Convert to BGR for marking in color
    marked_image = cv.cvtColor(image.astype(np.uint8), cv.COLOR_GRAY2BGR)

    if color == 'red':  # For hot pixels
        marked_image[image > threshold] = [0, 0, 255]
    elif color == 'blue':  # For dead pixels
        marked_image[image < threshold] = [255, 0, 0]

    # Save the marked image with a new name
    output_path = image_path.replace('.png', '_marked.png')
    cv.imwrite(output_path, marked_image)

# Mark hot pixels in the dark image and dead pixels in the white image
mark_pixels("dark_image.png", 50, 'red')
mark_pixels("Kamera2/changed/white_image.png", 50, 'blue')

# Correct the grayscale wedge image using the calculated dark and white images from the previous task
correct_image(
    "Kamera2/frame.png",
    "dark_image.png",
    white_image,
    "Kamera2/changed"
)

# Read the corrected image to evaluate the mean and standard deviation
corrected_image_path = "Kamera2/changed/corrected_image.png"
corrected_image = cv.imread(corrected_image_path, cv.IMREAD_GRAYSCALE)

if corrected_image is None:
    print(f"Error: Could not read the corrected image at {corrected_image_path}")
else:
    mean = np.mean(corrected_image)
    stddev = np.std(corrected_image)

    print(f'Corrected Image: Mean = {mean:.2f}, StdDev = {stddev:.2f}')

Corrected Image: Mean = 0.21, StdDev = 0.41
